<h1 style="text-align:center;"><center>Preprocessing for Radiomics (CCIDS)</center></h1>

## Contents
> 1. Get Image Array from Nifti File
> 2. Save Numpy Array to Nifti File
> 3. Resampling Voxel Size to (1, 1, 1)
> 4. Registration
> 5. White-stripe Normalization
> 6. N4 Bias Correction

---

## ※ Requirements

In [ ]:
!pip install nibabel
!pip install SimpleITK
!pip install scipy
!pip install statsmodels

## Step 1. Get Image Array from Nifti File

__- Use python package 'nibabel' to deal with nifti file(.nii)__

In [ ]:
import nibabel as nib

# Get Image Array (numpy array) from Nifti File
image_array = nib.load(file_path).get_data()

---

## Step 2. Save Numpy Array to Nifti File

__1) Nibabel__

In [ ]:
import numpy as np
import os

save_dir = '! directory to save nifti file !'
save_file_name = '! rename !.nii'

nifti_file = nib.Nifti1Image(image_array, np.eye(4))         # Make Nifti Object
nib.save(nifti_file, os.path.join(save_dir, save_file_name)) # Save

__2) Simple ITK__

In [ ]:
import os

save_dir = '! directory to save nifti file !'
save_file_name = '! rename !.nii'

ITK_image = sitk.GetImageFromArray(image_array)                    # Make Simple ITK Image Object
sitk.WriteImage(ITK_image, os.path.join(save_dir, save_file_name)) # Save

---

## Step 3. Resampling

In [ ]:
from utils.Resample import resample

# Get Origin Voxel Size from Nifti File.
origin_voxel_size = nib.load(file_path).header.get_zooms()

# Set Target Voxel Size
target_voxel_size = (1, 1, 1)

# Resampling
image_resampled = resample(image_array, origin_voxel_size, target_voxel_size)

---

## Step 4. Registration

__- We have to use 'SimpleITK' in registration process.__

In [ ]:
import SimpleITK as sitk

# (1) - Get ITK Image from Nifti File
image = sitk.ReadImage(file_path, sitk.sitkFloat32)

# (2) - Get ITK Image from Numpy Array
image = sitk.GetImageFromArray(image_array)

In [ ]:
from utils.Registration import initial_alignment, registration

# Image format is not numpy array but SimpleITK Image format
initial_transform, moving_resampled = initial_alignment(fixed_image, moving_image)
moving_resampled, final_transform = registration(fixed_image, moving_image, initial_transform)

# 'moving_resampled' is registered image. (SimpleITK Image format)

__- Registration of Mask Image (using 'final_transform')__

In [ ]:
# We have to use 'final_transform' to deal with mask image

resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed_image)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetDefaultPixelValue(0)
resampler.SetTransform(final_transform)

mask_registered = resampler.Execute(mask_image)

---

# Step 5. White-stripe Normalization

In [ ]:
from utils.WSNormalization import ws_normalize

contrast = '! contrast of sequence !' # 't1', 'flair', 'last' / 't2', 'largest' / 'md', 'first'

# If there is 'brain' mask file, put mask array to 'ws_normalize' function.
# Else, set 'mask_array' parameter to None (default is None). 
mask_array = nib.load(brain_mask_file_path).get_data()

normalized_array = ws_normalize(image_array, contrast, mask_array)

---

# Step 6. N4 Bias Correction

In [ ]:
from utils.N4Correction import n4correction

n4_corrected_array = n4correction(image_array)